In [2]:
!pip install datasets transformers pandasql beautifultable -qqq



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
import pandas as pd
import re

In [5]:
prefix = "translate english to sparql: "
model_name = model = "yazdipour/text-to-sparql-t5-base-qald9"

In [6]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, T5TokenizerFast
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = T5TokenizerFast.from_pretrained(model_name)
# tokenizer = T5TokenizerFast.from_pretrained(model_name, src_lang="en", tgt_lang="sparql")

tokenizer_config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [8]:
translator = pipeline(
    "translation_xx_to_yy",
    model=model,
    tokenizer=tokenizer,
    device=0 #0 for cuda, -1 for cpu
)

In [9]:
def replace_all(text, dict):
    for i, j in dict.items():
        text = text.replace(i, j)
    return text


In [25]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [13]:
!curl -X GET \
     "https://datasets-server.huggingface.co/rows?dataset=lc_quad&config=default&split=train&offset=0&length=100"

{"features":[{"feature_idx":0,"name":"NNQT_question","type":{"dtype":"string","_type":"Value"}},{"feature_idx":1,"name":"uid","type":{"dtype":"int32","_type":"Value"}},{"feature_idx":2,"name":"subgraph","type":{"dtype":"string","_type":"Value"}},{"feature_idx":3,"name":"template_index","type":{"dtype":"int32","_type":"Value"}},{"feature_idx":4,"name":"question","type":{"dtype":"string","_type":"Value"}},{"feature_idx":5,"name":"sparql_wikidata","type":{"dtype":"string","_type":"Value"}},{"feature_idx":6,"name":"sparql_dbpedia18","type":{"dtype":"string","_type":"Value"}},{"feature_idx":7,"name":"template","type":{"dtype":"string","_type":"Value"}},{"feature_idx":8,"name":"paraphrased_question","type":{"dtype":"string","_type":"Value"}}],"rows":[{"row_idx":0,"row":{"NNQT_question":"What is the {periodical literature} for {mouthpiece} of {Delta Air Lines}","uid":19719,"subgraph":"simple question right","template_index":65,"question":"What periodical literature does Delta Air Lines use as

In [26]:
df_q = pd.read_csv(f'/content/drive/My Drive/kdwd/q.csv')
df_p = pd.read_csv(f'/content/drive/My Drive/kdwd/p.csv')

In [27]:
def findID(pref, x):
    word = x.replace("_", " ").lower()
    try:
        if 'wd' != pref:
            # s = df_p[df_p.en.str.contains(word, na=False)]
            s = df_p[df_p['en']==word]
            if len(s)>0:
                id = s.iloc[0]['id']
                return 'P'+str(id)
        # Use else to do not replace missing Ps with Qs
        s = df_q[df_q['en']==word]
        # s = df_q[df_q.en.str.contains(word, na=False)]
        id = s.iloc[0]['id']
        return 'Q'+str(id)
    except: return x

In [28]:
findID('wd', 'instance')

'Q1664689'

In [29]:
def decode_props(qry):
    rep_dict = {}
    qry = qry.replace("[", "{").replace("]", "}")
    for m in re.finditer(":\w+", qry):
        pref = qry[m.start(0) - 3 : m.start(0)].strip()
        rep_dict[':'+m.group(0)[1:]] = ':'+findID(pref, m.group(0)[1:])
    return replace_all(qry, rep_dict)

In [30]:
decode_props(' select (COUNT(?sub) AS ?value ) [ ?sub wdt:grand-child wd:jacques_cousteau ]  ')


' select (COUNT(?sub) AS ?value ) { ?sub wdt:Q247153-child wd:Q83233 }  '

In [31]:
def translate(q):
    tr = translator(prefix+q, max_length=100)[0]['translation_text']
    return (tr, decode_props(tr))

In [32]:
from beautifultable import BeautifulTable
def pretty_translate(t,q):
    ans = translate(q)
    # print('QUESTION:\n' + q + '\nTARGET:\n'+ t +'\nRESULT:\n' + ans[0]+ '\nRESULT-DECODED:\n' + ans[1])
    table = BeautifulTable(maxwidth=140)
    # table.column_headers = ["", "Man Utd","Man City","T Hotspur"]
    table.rows.append(['QUESTION', q])
    table.rows.append(['Target', t])
    table.rows.append(['RawRESULT', ans[0]])
    table.rows.append(['RESULT', ans[1]])
    print(table)

In [33]:
prefix

'translate english to sparql: '

In [34]:
translator(prefix+'Who is Ronaldo R9?', max_length=100)

[{'translation_text': 'select distinct ?uri where [ ?uri dbo:Ronaldo_R9 dbo:res:Ronaldo_R9 ]'}]

In [35]:
translator(prefix+'Who is the player who played for Barcelona and InterMilan', max_length=100)

[{'translation_text': 'select distinct ?uri where [ ?uri dbo:member_of_sports_team dbo:Barcelona . ?uri dbo:member_of_sports_team dbr:InterMilan ]'}]

In [38]:
pretty_translate('select distinct ?uri where { ?uri dbo:member_of_sports_team dbo:Barcelona . ?uri dbo:member_of_sports_team dbr:InterMilan }', "Who's the player who played for Inter Milan and Barcelona?")


+-----------+-----------------------------------------------------------------------------------------------------------------------------+
| QUESTION  |                                 Who's the player who played for Inter Milan and Barcelona?                                  |
+-----------+-----------------------------------------------------------------------------------------------------------------------------+
|  Target   | select distinct ?uri where { ?uri dbo:member_of_sports_team dbo:Barcelona . ?uri dbo:member_of_sports_team dbr:InterMilan } |
+-----------+-----------------------------------------------------------------------------------------------------------------------------+
| RawRESULT |   select distinct ?uri where [ ?uri a dbo:member_of_sports_team dbo:member_of_sports_team dbr:Inter_Milan_and_Barcelona ]   |
+-----------+-----------------------------------------------------------------------------------------------------------------------------+
|  RESULT   |       